# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Important column names
COL_MOUSE_ID = "Mouse ID"
COL_MOUSE_SEX = "Sex"
COL_MOUSE_AGE = "Age_months"
COL_MOUSE_MASS = "Weight (g)"
COL_DRUG = "Drug Regimen"
COL_TIME = "Timepoint"
COL_TUMOR_VOL = "Tumor Volume (mm3)"
COL_METS = "Metastatic Sites"


In [12]:
# This function isn't required for the challenge, I just use it to get a feel for the data in a dataframe
def print_df_overview(df, title):        
    print(f"{title} DF Head:")
    print(df.head())
    print("----------------------------------")
    print(f"{title} DF Columns:")
    print(df.columns)
    print("----------------------------------")
    print(f"{title} DF dtypes:")
    print(df.dtypes)
    
    for col in df.columns:
        print("----------------------------------")
        print(f"{title} DF column value counts:")

        ABRIDGED_ROWS = 16
        counts = df[col].value_counts()
        if counts.size < 2*ABRIDGED_ROWS:
            # Print the counts all together
            print(col, "counts:")    
            print(counts)
        else:
            # Print top and bottom counts
            print(col, "top counts:")
            print(counts.iloc[0:ABRIDGED_ROWS])
            print(col, "bottom counts:")
            print(counts.iloc[-ABRIDGED_ROWS:])

In [13]:
# Read the mouse data and the study results
mice_df = pd.read_csv(mouse_metadata_path)
studies_df = pd.read_csv(study_results_path)

# For troubleshooting:
#print_df_overview(mice_df, "Mice")
#print_df_overview(studies_df, "Studies")

# Combine the data into a single DataFrame
combined_df = studies_df.merge(mice_df)

# Display the data table for preview
#print_df_overview(combined_df, "Combined")
combined_df

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,f932,0,45.000000,0,Ketapril,Male,15,29
2,g107,0,45.000000,0,Ketapril,Female,2,29
3,a457,0,45.000000,0,Ketapril,Female,11,30
4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...
1888,r944,45,41.581521,2,Capomulin,Male,12,25
1889,u364,45,31.023923,3,Capomulin,Male,18,17
1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1891,x773,45,58.634971,4,Placebo,Female,21,30


In [14]:
# Checking the number of mice.
combined_df[COL_MOUSE_ID].nunique()

249

In [15]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
# This returns the REDUNDANT entries, not all entries for duplicated mice:
redundant_df = combined_df.loc[combined_df.duplicated(subset = [COL_MOUSE_ID, COL_TIME]) ]
redundant_IDs = redundant_df[COL_MOUSE_ID].unique()
print(f"Mice with redundant study entries: {redundant_IDs}")

Mice with redundant study entries: ['g989']


In [16]:
# Optional: Get all the data for the duplicate mouse ID.
# This code assumes exactly one duplicated mouse, to more easily take advantageof Notebook's pretty formatting:
combined_df.loc[combined_df[COL_MOUSE_ID] == redundant_IDs[0]]

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
107,g989,0,45.000000,0,Propriva,Female,21,26
137,g989,0,45.000000,0,Propriva,Female,21,26
329,g989,5,48.786801,0,Propriva,Female,21,26
360,g989,5,47.570392,0,Propriva,Female,21,26
620,g989,10,51.745156,0,Propriva,Female,21,26
681,g989,10,49.880528,0,Propriva,Female,21,26
815,g989,15,51.325852,1,Propriva,Female,21,26
869,g989,15,53.442020,0,Propriva,Female,21,26
950,g989,20,55.326122,1,Propriva,Female,21,26
1111,g989,20,54.657650,1,Propriva,Female,21,26


In [17]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = combined_df.drop_duplicates(subset = [COL_MOUSE_ID, COL_TIME])

redundant_df = clean_df.loc[clean_df.duplicated(subset = [COL_MOUSE_ID, COL_TIME]) ]
redundant_IDs = redundant_df[COL_MOUSE_ID].unique()
print(f"Mice with redundant study entries (hopefully none): {redundant_IDs}")

clean_df

Mice with redundant study entries (hopefully none): []


,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,f932,0,45.000000,0,Ketapril,Male,15,29
2,g107,0,45.000000,0,Ketapril,Female,2,29
3,a457,0,45.000000,0,Ketapril,Female,11,30
4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...
1888,r944,45,41.581521,2,Capomulin,Male,12,25
1889,u364,45,31.023923,3,Capomulin,Male,18,17
1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1891,x773,45,58.634971,4,Placebo,Female,21,30


In [18]:
# Checking the number of mice in the clean DataFrame.
#
# The "starter" notebook seems to suggest that the correct output here is 248 but I don't think that is correct.
# You would get 248 mice if you removed ALL rows with for the particular mouse, but that isn't what's called for.
# There is no "duplicated mouse", there is a mouse with duplicated study entries. Removing rows for those duplicated entries
# does not remove the mouse. Mouse G989 had too many entries initially. I removed the redundant ones. Now G989 has the right amount.
# There have always been 249 mice.
clean_df[COL_MOUSE_ID].nunique()

249

## Summary Statistics

In [19]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen:
# mean, median, variance, standard deviation, and SEM of the tumor volume.
# Assemble the resulting series into a single summary DataFrame.


In [20]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [21]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.


In [22]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.


In [23]:
# Generate a pie chart, using Pandas, showing the distribution of unique female versus male mice used in the study

# Get the unique mice with their gender


# Make the pie chart


In [24]:
# Generate a pie chart, using pyplot, showing the distribution of unique female versus male mice used in the study

# Get the unique mice with their gender


# Make the pie chart


## Quartiles, Outliers and Boxplots

In [25]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [26]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers.


    # Locate the rows which contain mice on each drug and get the tumor volumes


    # add subset


    # Determine outliers using upper and lower bounds


In [27]:
# Generate a box plot that shows the distribution of the tumor volume for each treatment group.


## Line and Scatter Plots

In [28]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin


In [29]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen


## Correlation and Regression

In [30]:
# Calculate the correlation coefficient and a linear regression model
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
